## Importing and Cleaning

#### Running all the videos - classifying objects - and pickle object_id of vehicles and pedestrains

In [20]:
# This function allows the classification of moving objects into Pedestrians and Vehicles (and takes out all the other objects)
def classify(max_objects, df_positions):
    veh_id=[]
    ped_id=[]
    for i in range(max_objects):
        temp=df_positions.loc[df_positions['object_id'] == i].reset_index(drop=True)
        

        x_diff = (temp['x_coordinate'][temp.shape[0]-1]-temp['x_coordinate'][0])
        y_diff = (temp['y_coordinate'][temp.shape[0]-1]-temp['y_coordinate'][0])

    #If you want to print the differences for every object, uncomment the two lines below    
    # =============================================================================
    #     print('Diff in X for '+ str(i) + ' is ' +str(x_diff))
    #     print('Diff in Y for '+ str(i) + ' is ' +str(y_diff))
    #     print('\n\n')
    # =============================================================================


        if abs(x_diff) > 15:
            veh_id.append(i)
        elif abs(y_diff) > 2:
            ped_id.append(i)
        
    return veh_id, ped_id

#x,y=classify(max_objects, df_positions)
#print x,y

In [21]:
# This Functions converts the SQL databases into pandas and returns the database of |Object|Frame|X_positions|Y_positions
delta_t=1./30
def SQL_to_Pandas(filename):
    
    con = sqlite3.connect(path+filename)

    #Defining The number of objects we have
    sql_command = "SELECT COUNT(*) FROM objects"
    max_objects_sql = con.execute(sql_command)
    max_objects = max_objects_sql.fetchone()
    max_objects=max_objects[0]

    # Getting the positions of every object
    df_positions = pd.read_sql_query("SELECT a.object_id,  b.frame_number, x_coordinate, y_coordinate\
    FROM objects_features AS a \
    JOIN positions AS b ON a.trajectory_id = b.trajectory_id",con)

    # Merging the velocities and positions
    df = pd.DataFrame()

    df=df_positions.copy()
    
    df.columns = ['object', 'frame', 'x', 'y']
    
    # Compute the average of trajectories based on the features
    df=df.groupby(['object','frame']).mean().reset_index()
    
    #Dropping elements with nan
    df=df.dropna(axis=0, how='any')
    df_positions=df_positions.dropna(axis=0, how='any')
    
    return df, df_positions, max_objects

In [27]:
import sqlite3
from pandas.io.sql import read_sql
from sqlalchemy import create_engine
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#Get files from directory:
import os
import timeit
#=============================================================================
#=================================== ADD PATH TO THE SQLITE FILE =============
#=============================================================================
path = 'C:\\Users\\admin\\Desktop\\Capstone\\Final SQL\\revised_0206_1228\\'
#=============================================================================


#for loop to go over all the databases
for filename in os.listdir(path):   
    [df, df_positions, max_objects]=SQL_to_Pandas(filename)
    
    ## -------------- Plotting the trajectories -----------------
    for i in range(max_objects):
        X0=pd.Series.tolist((df.loc[df['object'] == i])['x'])
        Y0=pd.Series.tolist((df.loc[df['object'] == i])['y'])
        plt.scatter(X0,Y0)
        plt.xlabel("X positions")
        plt.ylabel("Y positions")
        plt.title("Trajectories of moving objects of"+ filename)
#     plt.show()

    ## --------------- Classification  --------------------------

    veh_id,ped_id=classify(max_objects,df_positions)

    ## Creating a dataframe with the id of the pedestrian and the vehicles
    ## use of -1 just to have the same length in dataframe

    len_max=max(len(veh_id), len(ped_id))
    veh_id_bis=[-1 for i in range(len_max)]
    ped_id_bis=[-1 for i in range(len_max)]
    for i in range(len(veh_id)):
        veh_id_bis[i]=veh_id[i]
    for i in range(len(ped_id)):
        ped_id_bis[i]=ped_id[i]
    #print veh_id_bis, ped_id_bis
    
    # --------------- Pickling the Pairing -----------------------
    
    pairing = {'veh_id': veh_id_bis, 'ped_id': ped_id_bis}
    #df_pairing = pd.DataFrame(data=pairing)
    #df_pairing.to_pickle("obj_ID_"+filename)


## Pairing output

In [28]:
## Another code was used in order to pair Pedestrians and Vehicles based on the pickled data from the previous part.
## As an input, 'pairs_---.csv' contains the paired interactions 

inter=pd.read_csv('pairs_0206_1228A.csv')
inter=inter.drop(['Unnamed: 0'], axis=1)
inter

,clip,ped_id,veh_id
0,clip_1,8,14
1,clip_2,28,23
2,clip_3,23,7
3,clip_4,15,10
4,clip_6,35,34
5,clip_7,32,17
6,clip_8,8,3
7,clip_8,8,7
8,clip_8,8,22
9,clip_8,25,22


## Computing the velocities and Filtering the signals

In [34]:
# Function to filter the velocities
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    import numpy as np
    from math import factorial
    window_size = np.abs(np.int(window_size))
    order = np.abs(np.int(order))
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
     # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [36]:
from scipy import signal
n = 5  # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1

k=0
len_inter=0
for i in range(len(inter.index)):
    if i!=5:
        [df, df_positions, max_objects]=SQL_to_Pandas(inter.iloc[i,0]+'.sqlite')

        veh_iden=int(inter.iloc[i,2])
        ped_iden=int(inter.iloc[i,1])

        vehicle=df.loc[df['object'] == veh_iden]    
        pedestrian=df.loc[df['object'] == ped_iden]

        interaction = pd.merge(vehicle, pedestrian, how ='inner', on = ['frame'])
        len_inter= len_inter+ len(interaction.index)
        interaction.columns = ['object1', 'frame', 'x1', 'y1', 'object2', 'x2', 'y2']
        
        
    # ============================================================== FIRST FILTER FOR POSITION
        b, a = signal.butter(2, 0.1)
        interaction['x1'] = signal.filtfilt(b,a,interaction['x1'])
        interaction['x2'] = signal.filtfilt(b,a,interaction['x2'])
        
        interaction['y1'] = signal.filtfilt(b,a,interaction['y1'])
        interaction['y2'] = signal.filtfilt(b,a,interaction['y2'])
        
    #--------------------------------------- Computing the speed of the objects
          
        interaction['v_x1']=(interaction['x1'].shift(-1)-interaction['x1'].shift(1))/(2*delta_t)
        interaction['v_y1']=(interaction['y1'].shift(-1)-interaction['y1'].shift(1))/(2*delta_t)

        interaction['v_x2']=(interaction['x2'].shift(-1)-interaction['x2'].shift(1))/(2*delta_t)
        interaction['v_y2']=(interaction['y2'].shift(-1)-interaction['y2'].shift(1))/(2*delta_t)      


    # ============================================================== FIRST FILTER FOR SPEED

        VX0=pd.Series.tolist(interaction['v_x1'])
        VY0=pd.Series.tolist(interaction['v_y1'])
        interaction['v_x1'] = savitzky_golay(np.array(VX0), len(VX0)/5, 2)
        interaction['v_y1'] = savitzky_golay(np.array(VY0), len(VY0)/5, 2)
        
        VX2=pd.Series.tolist(interaction['v_x2'])
        VY2=pd.Series.tolist(interaction['v_y2'])
        interaction['v_x2'] = savitzky_golay(np.array(VX2), 30, 2)
        interaction['v_y2'] = savitzky_golay(np.array(VY2), 30, 2)
        
    # --------------------------------- Grouping all interactions into one table
        interaction=interaction.dropna(axis=0, how='any')
        interaction=interaction[1:]
        interaction=interaction[:-1]
        if k==0:
            all_interactions=interaction.copy()
        else:
            frames = [all_interactions, interaction]
            all_interactions = pd.concat(frames)
        k=k+1

        # ------------------------------- Plotting the velocities
        X0=pd.Series.tolist(interaction['x1'])
        Y0=pd.Series.tolist(interaction['y1'])
        VX0=pd.Series.tolist(interaction['v_x1'])
        VY0=pd.Series.tolist(interaction['v_y1'])
        plt.subplot(1,2,1)
        plt.plot(X0)
        plt.subplot(1,2,2)
        plt.plot(VX0)
#         plt.show()

# -------------------------- Pickle all the interactions
# print interaction
# all_interactions.to_pickle("all_interactions")

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in absolute
  
C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in absolute
  from ipykernel import kernelapp as app


#### Careful, the velocities are not optimally filtered

## Features Engineering

### 1) Features:  Relative distances

In [38]:
# Calculate relative Distances and the target output Brake 
MLInput = pd.DataFrame()
MLInput['frame'] = all_interactions['frame']
MLInput['relative_X'] = all_interactions['x1'] - all_interactions['x2']
MLInput['relative_Y'] = all_interactions['y1'] - all_interactions['y2']
MLInput['relative_Vx'] = all_interactions['v_x1'] - all_interactions['v_x2']
MLInput['relative_Vy'] = all_interactions['v_y1'] - all_interactions['v_y2']
MLInput['brake'] = 0
#MLInput.columns = ['frame','relative_X','relative_Y','relative_Vx','relative_Vy','brake']

for frame in all_interactions['frame']:
    if len(all_interactions.loc[all_interactions['frame']==frame-1]['v_x1'].values)>0: 
        v_prev = all_interactions.loc[all_interactions['frame']==frame-1]['v_x1'].values[0]
        v = all_interactions.loc[all_interactions['frame']==frame]['v_x1'].values[0]
        if v - v_prev < 0:
            MLInput.loc[MLInput['frame']==frame,'brake'] = 1
        
print MLInput.head()
MLInput.to_csv("Input1")


    frame  relative_X  relative_Y  relative_Vx  relative_Vy  brake
16    276    6.946677   -3.685520     0.647138    -1.447542      1
17    277    6.975943   -3.738242     0.669243    -1.420924      0
18    278    7.004145   -3.789429     0.668854    -1.377635      0
19    279    7.030172   -3.838181     0.642549    -1.322921      0
20    280    7.053112   -3.883918     0.589705    -1.262429      1


##### Note that:  Input1 is a csv file used as input the the ML codes

In [39]:
MLInput.head()
print len(MLInput.index)

1860


### 2) Features:  Angles and Time to Pedestrian

In [40]:
delta_time=15 #seconds

MLInput2 = pd.DataFrame()
Intermediate = pd.DataFrame()

Intermediate['direction']=(MLInput['relative_Vx']>=0).astype(int)+ (MLInput['relative_Vx']<0).astype(int)*(-1)
Intermediate['X_r']=MLInput['relative_X']*Intermediate['direction'] # Check the direction though!
Intermediate['Y_r']=MLInput['relative_Y']*Intermediate['direction']

#Create the new features
MLInput2['alpha']=np.abs(np.arctan2(Intermediate['Y_r'],Intermediate['X_r']))*180/np.pi
MLInput2['delta_alpha']=(MLInput2['alpha'].shift(1)-MLInput2['alpha'])/delta_time
MLInput2['tt_ped']=abs(MLInput['relative_X']/all_interactions['v_x1'])
MLInput2['brake']=MLInput['brake']
MLInput2=MLInput2.dropna(axis=0, how='any')

#changing all the data for when the angle alpha is >90 degree
MLInput2['brake']=MLInput2['brake']*(MLInput2['alpha']<90).astype(int)
MLInput2.to_csv("Input2")
MLInput2

,alpha,delta_alpha,tt_ped,brake
17,28.185798,-0.015858,26.995592,0
18,28.414571,-0.015252,23.457957,0
19,28.632698,-0.014542,22.120663,0
20,28.840140,-0.013830,22.533274,1
21,29.038350,-0.013214,24.893850,1
22,29.230018,-0.012778,30.313526,1
23,29.418550,-0.012569,42.481803,1
24,29.607228,-0.012579,78.665626,1
25,29.798406,-0.012745,684.491183,1
26,150.006740,-8.013889,107.591415,0
